In [15]:
start_time = time.time()
reg = [g1.read_mb_register(i) for i in correct_mb_dict]
print((f'{time.time() - start_time:.1f} seconds'))

5.0 seconds


In [22]:
start_time = time.time()
g1.get_protections()
print((f'{time.time() - start_time:.1f} seconds'))

0.9 seconds


In [1]:
import doc_objects
import time
import config

# doc_objects.send_msg(723253749, 'Старт скрипта.')

g1 = doc_objects.Genset('/dev/ttyS0', 1)
g2 = doc_objects.Genset('/dev/ttyS0', 2)
g3 = doc_objects.Genset('/dev/ttyS0', 3)
g4 = doc_objects.Genset('/dev/ttyS0', 4)
g5 = doc_objects.Genset('/dev/ttyS0', 5)

doc = (g1, g2, g3, g4, g5)

In [2]:
start_time = time.time()
for g in doc:
    g.get_update()
print((f'{time.time() - start_time:.1f} seconds'))

13.6 seconds


In [4]:
set([v['Type'] for k, v in g4.modbus_table.items()])

{'Binary#1',
 'Binary#10',
 'Binary#11',
 'Binary#2',
 'Binary#3',
 'Binary#4',
 'Binary#42',
 'Binary#43',
 'Binary#44',
 'Binary#45',
 'Binary#5',
 'Binary#6',
 'Binary#7',
 'Binary#8',
 'Binary#9',
 'Integer',
 'Unsigned'}

In [7]:
[(v['Name'], v['Curr_val'], v['Dim']) for k, v in g2.modbus_table.items()]

[('BIN', '0b10000001101010', ''),
 ('BIN-1', '0b00000011100010', ''),
 ('BIN-2', '0b00000010000010', ''),
 ('BIN-3', '0b00000000001111', ''),
 ('BIN-4', '0b00000000000000', ''),
 ('BIN-5', '0b00000001010101', ''),
 ('BIN-6', '0b00000000101001', ''),
 ('BIN-7', '0b00000001010100', ''),
 ('BIN-8', '0b00000000001010', ''),
 ('BOUT', '0b01000001000000', ''),
 ('Ubat', 25.9, 'V'),
 ('CPU temp', 26.8, '°C'),
 ('Engine Temp', 48.0, '°C'),
 ('Oil Pressure', 0.0, 'bar'),
 ('Oil Temp', 29.0, '°C'),
 ('Manifold Temp', 31.0, '°C'),
 ('Manifold Press', 2.2, 'bar'),
 ('Actual Timing', 0.0, '°'),
 ('Desired Timing', 0.0, '°'),
 ('Detonation', 0.0, ''),
 ('Cylinder 1', 36.0, '°C'),
 ('Cylinder 2', 36.0, '°C'),
 ('Cylinder 3', 37.0, '°C'),
 ('Cylinder 4', 35.0, '°C'),
 ('Cylinder 5', 36.0, '°C'),
 ('Cylinder 6', 36.0, '°C'),
 ('Cylinder 7', 37.0, '°C'),
 ('Cylinder 8', 35.0, '°C'),
 ('Cylinder 9', 37.0, '°C'),
 ('Cylinder 10', 35.0, '°C'),
 ('Cylinder 11', 35.0, '°C'),
 ('Cylinder 12', 35.0, '°C'),
 ('

In [9]:
g = g4

def _formating_register(adr, reg):
    # Если бинарное число, то возвращаю в двоичном виде
    if g.modbus_table[adr]['Type'].lower()[:3] == 'bin':
        return format(reg, '#016b')
    # Отрицательное число определяю по косвенным признакам
    elif (not g.modbus_table[adr]['Min'].isdigit()) and len(g.modbus_table[adr]['Min'])-1 and g.modbus_table[adr]['Min'][-1]!='*' and reg&0b1000000000000000:
        reg = reg - 65535
    # Десятичная точка
    dec = g.modbus_table[adr]['Dec']
    if dec!='-':
        reg = reg/10**int(dec)
    return reg
    
    
    
def get_update():
    chank_arr = g.get_chunk_intervals(list(g.modbus_table.keys()))
    for chank in chank_arr:
        regs = g.read_mb_registers(chank[0], chank[1])
        if regs:
            for c in range(len(regs)):
                adr = chank[0] + c
                g.modbus_table[adr]['Prev_val'] = g.modbus_table[adr]['Curr_val']
                g.modbus_table[adr]['Curr_val'] = _formating_register(adr, regs[c])
    return None

start_time = time.time()
get_update()
print((f'{time.time() - start_time:.1f} seconds'))

4.5 seconds


In [31]:
format(3, '#016b')
g4.modbus_table[5]['Type'].lower()[:3] == 'bin'

'bin'

У меня есть массив данных, вся таблица регистров, но мне они все не нужны. Есть критерии для опроса
Надо опрашивать, только то что надо, что не надо опрашивать не надо
long опрашиваю отдельно, их не надо в пакетный опрос включать
Должны быть две группы параметров, для регулярного внесения в БД и для внесения по изменению (это статические параметры)

In [12]:
config.values_if_changed + config.regular_values

('Analog protect',
 'Basic settings',
 'Comms settings',
 'Engine params',
 'Engine protect',
 'Gener protect',
 'Load shedding',
 'Mains protect',
 'PLC',
 'ProcessControl',
 'Pwr management',
 'Sync/Load ctrl',
 'Volt/PF ctrl',
 'Analog CU',
 'AnalogInputs 1',
 'AnalogInputs 2',
 'AnalogInputs 3',
 'AnalogInputs 4',
 'AnalogInputs 5',
 'Bus values',
 'Engine values',
 'Gener values',
 'Bin inputs CU',
 'Bin outputs CU',
 'Binary Inputs',
 'Binary Outputs')

In [17]:
import config
len({(v['Name'], v['Curr_val']) for k, v in g1.modbus_table.items() if v['Group'] in (config.values_if_changed + config.regular_values) and v['Name'] not in (config.val_ignore_list)})

443

In [9]:
start_time = time.time()

for adr in g1.modbus_table:
    # Проверяю условия:
    # если 'Name' в списке игнорируемых, то пропускаю цикл
    if g1.modbus_table[adr]['Name'] in config.val_ignore_list:
        continue
    # если 'Name' в списке для read_long, то читаю функцией read_long
    elif g1.modbus_table[adr]['Name'] in config.var_long_list:
        g1.modbus_table[adr]['Curr_val'] = g1.read_mb_long(adr)
        continue
    # если max<0 то signed
#     elif (not g1.modbus_table[adr]['Min'].isdigit()) and len(g1.modbus_table[adr]['Min'])-1 and g1.modbus_table[adr]['Min'][-1]!='*':
#         g1.modbus_table[adr]['Curr_val'] = int(g1.read_mb_register(adr))-65535
    # если 'Type' is 'bin', то записываю в двоичном виде
    g1.modbus_table[adr]['Curr_val'] = g1.read_mb_register(adr)
    
print((f'{time.time() - start_time:.1f} seconds'))

21.1 seconds


In [9]:
g1.read_mb_long(3594)

5729234

In [2]:
print(f'ГПГУ{g1.address}, наработка: {g1.read_long(3586)} мч')
print(f'ГПГУ{g2.address}, наработка: {g2.read_long(3586)} мч')
print(f'ГПГУ{g3.address}, наработка: {g3.read_mb_registers(3821, 2)} мч')
print(f'ГПГУ{g4.address}, наработка: {g4.read_mb_registers(3821, 2)} мч')
print(f'ГПГУ{g5.address}, наработка: {g5.read_mb_registers(3821, 2)} мч')

ГПГУ1, наработка: 8469 мч
ГПГУ2, наработка: 7930 мч
ГПГУ3, наработка: None мч
ГПГУ4, наработка: [0, 9249] мч
ГПГУ5, наработка: None мч


In [ ]:
ГПГУ1, наработка: 8461 мч
ГПГУ2, наработка: 7930 мч
ГПГУ3, наработка: None мч
ГПГУ4, наработка: [0, 9241] мч
ГПГУ5, наработка: None мч

In [5]:
import telebot
token = '1325955552:AAHQJzvhu-IhcMpsVGX5dUsfXTJ7fFECoJs'

tb = telebot.TeleBot(token)